In [2]:
import urllib2
from bs4 import BeautifulSoup
import re
import pandas as pd
from random import randint
from time import sleep
from sklearn.linear_model import LinearRegression
from patsy import dmatrices
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [4]:
def top100_soup(num):
    '''pull in url and create soup obj'''
    yr = str(num)
    url = 'http://www.boxofficemojo.com/yearly/chart/?yr='+ yr + '&p=.htm'
    page = urllib2.urlopen(url)
    soup = BeautifulSoup(page)
    return soup


In [5]:
def remainder_soup(c,years):
    n = str(c)
    yr = str(years)
    sleep(randint(10,100))
    url = 'http://www.boxofficemojo.com/yearly/chart/?page=' + n +'&view=releasedate&view2=domestic&yr=' + yr + '&p=htm'
    page = urllib2.urlopen(url)
    soup = BeautifulSoup(page)
    return soup


In [6]:
def top_movies_urls(soup):
    '''pull <a href line from soup and formatt url'''
    all_movies = []
    all_urls = []
    for link in soup.find('table', {'cellpadding':'5'}).find_all('a', href = True):
        if "/movies/?id=" in link['href']:
            all_movies.append(link)
            
    for element in all_movies:
        test1 = str(element).split("=",1)
        test2 = test1[1].split(">")
        url = ("http://www.boxofficemojo.com" + test2[0]).replace('"','')
        all_urls.append(url)
    return all_urls


In [7]:
def get_movie_value(soup, field_name):
    """takes a string attribute of a movie on the movie, and returns the 
    string in the next sibling object"""
    obj = soup.find(text = re.compile(field_name))
    if not obj:
        return None
    next_sibling = obj.findNextSibling()
    if next_sibling:
        return next_sibling.text
    else:
        return None

In [8]:
def get_values(soup, field_names):
    '''takes a list of values and c'''
    field_values = []
    for f in field_names:
        sib_val = get_movie_value(soup, f)
        field_values.append(sib_val)
    return field_values

In [9]:
def title_string(soup):
    '''pulls out the title of the film'''
    title = soup.find("title").text
    title = title.split("(")[0].strip()
    return title

In [10]:
#find domestic total

def domestic_tot(soup):
    dom_tot = soup.find('table', {'cellpadding':'4'}).find('b')
    dom_tot = dom_tot.text
    dom_tot = dom_tot.replace('$', "")
    #print type(dom_tot)
    return dom_tot

#domestic_tot(soup)

In [11]:
def movie_data(urls):
    headers = ["Release Date", "Genre:", "MPAA Rating","Budget"]
    movie_data = []

    for links in urls:
        movie_data_row = []
        page = urllib2.urlopen(links)
        soup = BeautifulSoup(page)
        movie_data_row.append(title_string(soup))
        movie_data_row.append(domestic_tot(soup))
        movie_data_row.extend(get_values(soup, headers))
        movie_data.append(movie_data_row)
    return movie_data


In [16]:
years = [2010,2011,2012,2013,2014]
#years = [2010]
count = [2,3,4,5,6]
#count = [2]
final_list = []
for y in years:
    sleep(randint(10,100))
    soup = top100_soup(y)
    urls = top_movies_urls(soup)
    final_list.extend(movie_data(urls))
    print "finished top for year"
    for c in count:
        print "printing remainder of year", c
        sleep(randint(10,100))
        soup = remainder_soup(c,y)
        print "finished soup", c
        urls = top_movies_urls(soup)
        print "finished urls for ",c
        final_list.extend(movie_data(urls))
        print "appended final list"

finished top for year
printing remainder of year 2
finished soup 2
finished urls for  2
appended final list
printing remainder of year 3
finished soup 3
finished urls for  3
appended final list
printing remainder of year 4
finished soup 4
finished urls for  4
appended final list
printing remainder of year 5
finished soup 5
finished urls for  5
appended final list
printing remainder of year 6
finished soup 6
finished urls for  6
appended final list
finished top for year
printing remainder of year 2
finished soup 2
finished urls for  2
appended final list
printing remainder of year 3
finished soup 3
finished urls for  3
appended final list
printing remainder of year 4
finished soup 4
finished urls for  4
appended final list
printing remainder of year 5
finished soup 5
finished urls for  5
appended final list
printing remainder of year 6
finished soup 6
finished urls for  6
appended final list
finished top for year
printing remainder of year 2
finished soup 2
finished urls for  2
appended

In [19]:
print final_list

[[u'Toy Story 3', u'415,004,880', u'June 18, 2010', u'Animation', u'G', u'$200 million'], [u'Alice in Wonderland', u'334,191,110', u'March 5, 2010', u'Family Adventure', u'PG', u'$200 million'], [u'Iron Man 2', u'312,433,331', u'May 7, 2010', u'Action / Adventure', u'PG-13', u'$200 million'], [u'The Twilight Saga: Eclipse', u'300,531,751', u'June 30, 2010', u'Romance', u'PG-13', u'$68 million'], [u'Harry Potter and the Deathly Hallows Part 1', u'295,983,305', u'November 19, 2010', u'Fantasy', u'PG-13', u'N/A'], [u'Inception', u'292,576,195', u'July 16, 2010', u'Sci-Fi Action', u'PG-13', u'$160 million'], [u'Despicable Me', u'251,513,985', u'July 9, 2010', u'Animation', u'PG', u'$69 million'], [u'Shrek Forever After', u'238,736,787', u'May 21, 2010', u'Animation', u'PG', u'$165 million'], [u'How to Train Your Dragon', u'217,581,231', u'March 26, 2010', u'Animation', u'PG', u'$165 million'], [u'Tangled', u'200,821,936', u'November 24, 2010', u'Animation', u'PG', u'$260 million'], [u'The 

In [21]:
cols = headers2 = ("Title","Domestic Total", "Release Date", "Genre", "MPAA Rating","Budget")
df = pd.DataFrame(final_list)
df.columns = (cols)
df

,Title,Domestic Total,Release Date,Genre,MPAA Rating,Budget
0,Toy Story 3,"415,004,880","June 18, 2010",Animation,G,$200 million
1,Alice in Wonderland,"334,191,110","March 5, 2010",Family Adventure,PG,$200 million
2,Iron Man 2,"312,433,331","May 7, 2010",Action / Adventure,PG-13,$200 million
3,The Twilight Saga: Eclipse,"300,531,751","June 30, 2010",Romance,PG-13,$68 million
4,Harry Potter and the Deathly Hallows Part 1,"295,983,305","November 19, 2010",Fantasy,PG-13,N/A
5,Inception,"292,576,195","July 16, 2010",Sci-Fi Action,PG-13,$160 million
6,Despicable Me,"251,513,985","July 9, 2010",Animation,PG,$69 million
7,Shrek Forever After,"238,736,787","May 21, 2010",Animation,PG,$165 million
8,How to Train Your Dragon,"217,581,231","March 26, 2010",Animation,PG,$165 million
9,Tangled,"200,821,936","November 24, 2010",Animation,PG,$260 million


In [22]:
df.to_csv("movies2.csv")